## Competitions: Kaggle Season 4 Episode 1 - Binary Classification with a Bank Churn Dataset 
https://www.kaggle.com/competitions/playground-series-s4e1/data

### Imports and Read Data

In [2]:
import pandas as pd
import numpy as np
import sklearn

In [11]:
TRAIN = pd.read_csv("./data/train.csv", na_values='NaN')
TEST = pd.read_csv("./data/test.csv", na_values='NaN')
SAMPLE_SUBMISSION = pd.read_csv('./data/sample_submission.csv')

In [12]:
TEST.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,165034,15773898,Lucchese,586,France,Female,23.00,2,0.00,2,0.00,1.00,"160,976.75"
1,165035,15782418,Nott,683,France,Female,46.00,2,0.00,1,1.00,0.00,"72,549.27"
2,165036,15807120,K?,656,France,Female,34.00,7,0.00,2,1.00,0.00,"138,882.09"
3,165037,15808905,O'Donnell,681,France,Male,36.00,8,0.00,1,1.00,0.00,"113,931.57"
4,165038,15607314,Higgins,752,Germany,Male,38.00,10,"121,263.62",1,1.00,0.00,"139,431.00"


In [13]:

TRAIN.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.00,3,0.00,2,1.00,0.00,"181,449.97",0
1,1,15749177,Okwudiliolisa,627,France,Male,33.00,1,0.00,2,1.00,1.00,"49,503.50",0
2,2,15694510,Hsueh,678,France,Male,40.00,10,0.00,2,1.00,0.00,"184,866.69",0
3,3,15741417,Kao,581,France,Male,34.00,2,"148,882.54",1,1.00,1.00,"84,560.88",0
4,4,15766172,Chiemenam,716,Spain,Male,33.00,5,0.00,2,1.00,1.00,"15,068.83",0


### Basic Data Exploration

NOTES:
- No missing data
- 3 objects: Surname (2797 Surnames), Gender (Male, Female), Geography (France, Germany, Spain)
- 2 IDs: CustomerID and ID (There are about 8 times more IDs than customer IDs (Based on Tenure?))

In [14]:
pd.options.display.float_format = '{:,.2f}'.format
def summary(df):
    print(f'data shape: {df.shape}')
    summ = pd.DataFrame(df.dtypes, columns=['data type'])
    summ['#missing'] = df.isnull().sum().values 
    summ['%missing'] = df.isnull().sum().values / len(df) * 100
    summ['#unique'] = df.nunique().values
    desc = pd.DataFrame(df.describe(include='all').transpose())
    summ['min'] = desc['min'].values
    summ['max'] = desc['max'].values
    summ['average'] = desc['mean'].values
    summ['standard_deviation'] = desc['std'].values
    summ['first value'] = df.loc[0].values
    summ['second value'] = df.loc[1].values
    summ['third value'] = df.loc[2].values
    
    return summ

summary(TRAIN)

data shape: (165034, 14)


,data type,#missing,%missing,#unique,min,max,average,standard_deviation,first value,second value,third value
id,int64,0,0.00,165034,0.00,"165,033.00","82,516.50","47,641.36",0,1,2
CustomerId,int64,0,0.00,23221,"15,565,701.00","15,815,690.00","15,692,005.02","71,397.82",15674932,15749177,15694510
Surname,object,0,0.00,2797,NaN,NaN,NaN,NaN,Okwudilichukwu,Okwudiliolisa,Hsueh
CreditScore,int64,0,0.00,457,350.00,850.00,656.45,80.10,668,627,678
Geography,object,0,0.00,3,NaN,NaN,NaN,NaN,France,France,France
Gender,object,0,0.00,2,NaN,NaN,NaN,NaN,Male,Male,Male
Age,float64,0,0.00,71,18.00,92.00,38.13,8.87,33.00,33.00,40.00
Tenure,int64,0,0.00,11,0.00,10.00,5.02,2.81,3,1,10
Balance,float64,0,0.00,30075,0.00,"250,898.09","55,478.09","62,817.66",0.00,0.00,0.00
NumOfProducts,int64,0,0.00,4,1.00,4.00,1.55,0.55,2,2,2


### Data Cleaning

In [33]:
train = TRAIN.copy()
train.drop(columns=['id'], inplace=True)
train.head()
dummies = pd.get_dummies(train[['Gender', 'Geography']], prefix='', prefix_sep='')
train_d = pd.concat([train, dummies], axis=1)
test = train_d.sort_values(by=['CustomerId', 'Tenure'])
test.head()
t = train_d.groupby(by=['CustomerId']).size().reset_index(name='count')
t.sort_values(by=['count', 'CustomerId'])

,CustomerId,count
3,15565759,1
7,15565807,1
10,15565891,1
12,15566028,1
18,15566187,1
...,...,...
21154,15793331,90
2953,15595588,91
1775,15585835,98
241,15570194,99


In [30]:
test.loc[test.CustomerId == 15682355, :]

,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,...,Zubarev,Zubareva,Zuev,Zuyev,Zuyeva,France,Germany,Spain,Female,Male
20482,15682355,663,35.00,0,0.00,1,1.00,0.00,"138,539.73",1,...,0,0,0,0,0,1,0,0,0,1
24598,15682355,813,33.00,0,"117,419.35",1,0.00,1.00,"120,736.04",0,...,0,0,0,0,0,0,1,0,0,1
35178,15682355,717,37.00,0,0.00,1,1.00,0.00,"162,697.54",1,...,0,0,0,0,0,1,0,0,1,0
5063,15682355,790,44.00,1,"128,100.75",1,1.00,0.00,"164,517.96",1,...,0,0,0,0,0,0,0,1,0,1
63843,15682355,603,27.00,1,"131,647.01",1,1.00,0.00,"88,890.05",0,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138764,15682355,805,37.00,9,"92,855.02",2,1.00,0.00,"92,985.78",1,...,0,0,0,0,0,0,1,0,1,0
147394,15682355,553,51.00,9,"132,310.39",1,1.00,1.00,"92,934.93",1,...,0,0,0,0,0,0,0,1,1,0
154632,15682355,593,40.00,9,0.00,2,0.00,1.00,"119,882.91",0,...,0,0,0,0,0,0,0,1,1,0
30924,15682355,710,42.00,10,"138,053.97",1,1.00,0.00,"114,957.85",0,...,0,0,0,0,0,1,0,0,1,0


### Data Model

LSTM Model? (Something with memory to use based on Tenure?)